# <img style="float: left; padding-right: 10px; width: 150px" src="../../imagini/unitbv2.png"> SIIPA2020- Știința Explorării și Exploatării Datelor - ADS1  






## <center>Curs 10 - Clasificatori ML - exemple</center>
**Universitatea Transilvania**<br>
**Semestrul I, 2020-2021**<br>
**Instructor:** Conf. univ. dr. Teodor &Scedil;tefan B&icirc;ldea<br>
**Contact:** teodor.bildea@unitbv.ro<br>


# Curprins:

1. Exemplu pas cu pas de antrenare clasificator Random Forest cu Grid Search pentru parametrii
2. Exemplu folosind GridSearchCV

Pentru eșantionare stratificată avem diverse opțiuni (https://scikit-learn.org/stable/modules/cross_validation.html#stratification).

Am optat pentru:
- train_test_split cu parametrul <b>stratify</b> setat pe coloana variabilei răspuns cu clasele.
- StratifiedShuffleSplit

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# incarcare set de date
pima = pd.read_csv("pima-indians-diabetes.csv", header=0, names=col_names)

# 
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']



# instantiere clasa de esantionare stratificata
train_test_selector = StratifiedShuffleSplit(n_splits =1, test_size = 0.3, random_state = 77)

# bucla prin valorile variabilei target (in cazul nostru binare) pentru a construi seturile de train, test
for train_idx, test_idx in train_test_selector.split(pima,pima['label']):
    train_set = pima.loc[train_idx]
    test_set = pima.loc[test_idx]

print(train_set.shape[0] + test_set.shape[0], pima.shape[0])
display(train_set.head(5))
print('Proportie clase in setul initial: clasa 0 -', pima[pima.label ==0].shape[0]/pima.shape[0] )
print('Proportie clase in setul de antrenament: clasa 0 -', train_set[train_set.label ==0].shape[0]/train_set.shape[0] )
print('Proportie clase in setul de testare: clasa 0 - ', test_set[test_set.label ==0].shape[0]/test_set.shape[0] )

768 768


,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
714,3,102,74,0,0,29.5,0.121,32,0
328,2,102,86,36,120,45.5,0.127,23,1
530,2,122,60,18,106,29.8,0.717,22,0
4,0,137,40,35,168,43.1,2.288,33,1
389,3,100,68,23,81,31.6,0.949,28,0


Proportie clase in setul initial: clasa 0 - 0.6510416666666666
Proportie clase in setul de antrenament: clasa 0 - 0.6517690875232774
Proportie clase in setul de testare: clasa 0 -  0.6493506493506493


In [49]:

# Split dataset into training set and test set
X_train, X_test = train_test_split(pima,test_size=0.3, stratify = pima[target_col]) # 70% training and 30% test

In [50]:
print('Proportie clase in setul initial: clasa 0 -', pima[pima.label ==0].shape[0]/pima.shape[0] )
print('Proportie clase in setul de antrenament: clasa 0 -', X_train[X_train.label ==0].shape[0]/X_train.shape[0] )
print('Proportie clase in setul de testare: clasa 0 - ', X_test[X_test.label ==0].shape[0]/X_test.shape[0] )

Proportie clase in setul initial: clasa 0 - 0.6510416666666666
Proportie clase in setul de antrenament: clasa 0 - 0.6517690875232774
Proportie clase in setul de testare: clasa 0 -  0.6493506493506493


# Exemplu pas cu pas de antrenare clasificator Random Forest cu Grid Search pentru parametrii

Codul de mai jos este validare incrucisata facuta de la zero. Nu beneficiaza de facilitati de paralelizare.
Veti vedea ulterior acelasi demers intr-o implementare scikitlearn cu paralelilzare.


In [55]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as accuracy
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier


def antrenament(X, target_col, bucati_validare_incrucisata=10):
    
    # instantiere clasa de esantionare stratificata - alege spargerea 10 pentru validare încrucișată
    train_test_selector = StratifiedShuffleSplit(n_splits =bucati_validare_incrucisata,
                                                 test_size = 0.3)#, random_state = 77)

    # pentru rezutate
    res_test = []
    param_grid = {
                    'n_estimators': [100, 200, 500],
                    'max_features': ['auto'],
                    'max_samples' : [0.1, 0.2, 0.3, 0.4, 0.5] ,
                    'criterion': ['gini','entropy'],
                 }
    
   
    for n_estimators,max_features,max_samples, criterion in product(param_grid['n_estimators'],
                                                                    param_grid['max_features'],
                                                                    param_grid['max_samples'],
                                                                    param_grid['criterion']):
                
        print( n_estimators,max_features,max_samples, criterion)
        i = 0
        # enumerare validări încrucișate
        for train_idx, test_idx in train_test_selector.split(X, X[target_col]):

            # preparare set antrenament local
            Xtrain = X.iloc[train_idx][ [c for c in X.columns if c!= target_col]]
            
            ###
            # ---- daca aveti atribute care depind de tot setul de antrenament, aici se construiesc parametrii
            # necesari si se genereaza atributele rezultate pe Xtrain 
            ##
            
            # preparare set date test local
            Xtest = X.iloc[test_idx][[c for c in X.columns if c!= target_col]]
            #
            #  --- aici s-ar genera atributele derivate pentru Xtest, folosind parametrii rezultati din Xtrain
            #
            
            
            # preparere variabila răspuns
            ytrain = X.iloc[train_idx][target_col]
            ytest = X.iloc[test_idx][target_col]

            model = rfc=RandomForestClassifier(
                n_estimators = n_estimators,
                max_features = max_features,
                max_samples = max_samples,
                criterion = criterion,
                random_state=42)



            model.fit(Xtrain, ytrain)

            # fitting error
            y_fit_reg = model.predict(Xtrain)
            a=accuracy(y_true = ytrain, y_pred =  y_fit_reg)


            # error on test - unseen data
            y_pred_reg = model.predict(Xtest)

            b=accuracy(y_true = ytest, y_pred =  y_pred_reg)

            res_test.append({"n_estimators":n_estimators,
                             "max_features":max_features, 
                             "max_samples":max_samples,
                             "criterion":criterion,
                             "fitting_scores":a,
                            "prediction_scores":b,
                            "model":model})

            print("Runda antrenare ",i,"scor partial predictie: ",b )
            i = i + 1
                        
    return pd.DataFrame(res_test).sort_values(by = 'prediction_scores', ascending = False).loc[0].to_dict()  
        
results = []        
# cateva iteratii 
max_iter = 5

# coloana cu variabila raspuns
for i in range(max_iter):
    
    # Split dataset into training set and test set
    X_train, X_test = train_test_split(pima,test_size=0.3, stratify = pima[target_col]) # 70% training and 30% test
    #X_train, X_test,y_train, y_test = train_test_split(pima, pima[target_col],test_size=0.3) # 70% training and 30% test
    
    # nu folosesc y_train si y_test; voi lua valorile din X_train dupa spargerea din CV - asta ca 
    # sa arat cum s-ar faca daca aveti atribute derivate din intregul esantion
    
    model_row = antrenament(X_train, target_col,bucati_validare_incrucisata = 5)
    
    final_model = rfc=RandomForestClassifier(
                            n_estimators = model_row['n_estimators'],
                            max_features = model_row['max_features'],
                            max_samples = model_row['max_samples'],
                            criterion = model_row['criterion'])
    
    y_train = X_train[target_col]
    final_model.fit(X_train[[c for c in pima.columns if c!= target_col]], y_train)
    y_train_pred = final_model.predict(X_train[[c for c in pima.columns if c!= target_col]])
    y_test_pred = final_model.predict(X_test[[c for c in pima.columns if c!= target_col]])
    model_row['final_test_fitting_score']= accuracy(y_true = y_train, y_pred=y_train_pred)
    model_row['final_test_prediction_score']= accuracy(y_true = y_test, y_pred=y_test_pred)
    results.append(model_row)
    

df_final = pd.DataFrame(results)
display(df_final[['final_test_fitting_score','final_test_prediction_score','n_estimators','max_features','max_samples','criterion']])

100 auto 0.1 gini
Runda antrenare  0 scor partial predictie:  0.7839506172839507
Runda antrenare  1 scor partial predictie:  0.8148148148148148
Runda antrenare  2 scor partial predictie:  0.7777777777777778
Runda antrenare  3 scor partial predictie:  0.8024691358024691
Runda antrenare  4 scor partial predictie:  0.7654320987654321
100 auto 0.1 entropy
Runda antrenare  0 scor partial predictie:  0.7901234567901234
Runda antrenare  1 scor partial predictie:  0.7530864197530864
Runda antrenare  2 scor partial predictie:  0.7654320987654321
Runda antrenare  3 scor partial predictie:  0.7530864197530864
Runda antrenare  4 scor partial predictie:  0.7407407407407407
100 auto 0.2 gini
Runda antrenare  0 scor partial predictie:  0.7469135802469136
Runda antrenare  1 scor partial predictie:  0.7530864197530864
Runda antrenare  2 scor partial predictie:  0.7407407407407407
Runda antrenare  3 scor partial predictie:  0.8518518518518519
Runda antrenare  4 scor partial predictie:  0.790123456790123

Runda antrenare  3 scor partial predictie:  0.7592592592592593
Runda antrenare  4 scor partial predictie:  0.7592592592592593
500 auto 0.3 entropy
Runda antrenare  0 scor partial predictie:  0.7469135802469136
Runda antrenare  1 scor partial predictie:  0.808641975308642
Runda antrenare  2 scor partial predictie:  0.7222222222222222
Runda antrenare  3 scor partial predictie:  0.7962962962962963
Runda antrenare  4 scor partial predictie:  0.7962962962962963
500 auto 0.4 gini
Runda antrenare  0 scor partial predictie:  0.7592592592592593
Runda antrenare  1 scor partial predictie:  0.7530864197530864
Runda antrenare  2 scor partial predictie:  0.7407407407407407
Runda antrenare  3 scor partial predictie:  0.7962962962962963
Runda antrenare  4 scor partial predictie:  0.7716049382716049
500 auto 0.4 entropy
Runda antrenare  0 scor partial predictie:  0.8271604938271605
Runda antrenare  1 scor partial predictie:  0.7530864197530864
Runda antrenare  2 scor partial predictie:  0.7222222222222

Runda antrenare  1 scor partial predictie:  0.7407407407407407
Runda antrenare  2 scor partial predictie:  0.8024691358024691
Runda antrenare  3 scor partial predictie:  0.7839506172839507
Runda antrenare  4 scor partial predictie:  0.7592592592592593
500 auto 0.1 gini
Runda antrenare  0 scor partial predictie:  0.7530864197530864
Runda antrenare  1 scor partial predictie:  0.7469135802469136
Runda antrenare  2 scor partial predictie:  0.7839506172839507
Runda antrenare  3 scor partial predictie:  0.7592592592592593
Runda antrenare  4 scor partial predictie:  0.808641975308642
500 auto 0.1 entropy
Runda antrenare  0 scor partial predictie:  0.7716049382716049
Runda antrenare  1 scor partial predictie:  0.8271604938271605
Runda antrenare  2 scor partial predictie:  0.8148148148148148
Runda antrenare  3 scor partial predictie:  0.7530864197530864
Runda antrenare  4 scor partial predictie:  0.7777777777777778
500 auto 0.2 gini
Runda antrenare  0 scor partial predictie:  0.7777777777777778

Runda antrenare  4 scor partial predictie:  0.7469135802469136
200 auto 0.3 gini
Runda antrenare  0 scor partial predictie:  0.7839506172839507
Runda antrenare  1 scor partial predictie:  0.7716049382716049
Runda antrenare  2 scor partial predictie:  0.7530864197530864
Runda antrenare  3 scor partial predictie:  0.7962962962962963
Runda antrenare  4 scor partial predictie:  0.7901234567901234
200 auto 0.3 entropy
Runda antrenare  0 scor partial predictie:  0.7469135802469136
Runda antrenare  1 scor partial predictie:  0.7716049382716049
Runda antrenare  2 scor partial predictie:  0.7777777777777778
Runda antrenare  3 scor partial predictie:  0.7716049382716049
Runda antrenare  4 scor partial predictie:  0.7716049382716049
200 auto 0.4 gini
Runda antrenare  0 scor partial predictie:  0.7716049382716049
Runda antrenare  1 scor partial predictie:  0.7716049382716049
Runda antrenare  2 scor partial predictie:  0.7654320987654321
Runda antrenare  3 scor partial predictie:  0.734567901234567

Runda antrenare  2 scor partial predictie:  0.7962962962962963
Runda antrenare  3 scor partial predictie:  0.7469135802469136
Runda antrenare  4 scor partial predictie:  0.7037037037037037
100 auto 0.5 entropy
Runda antrenare  0 scor partial predictie:  0.7839506172839507
Runda antrenare  1 scor partial predictie:  0.7901234567901234
Runda antrenare  2 scor partial predictie:  0.7530864197530864
Runda antrenare  3 scor partial predictie:  0.8024691358024691
Runda antrenare  4 scor partial predictie:  0.7962962962962963
200 auto 0.1 gini
Runda antrenare  0 scor partial predictie:  0.7901234567901234
Runda antrenare  1 scor partial predictie:  0.7716049382716049
Runda antrenare  2 scor partial predictie:  0.7777777777777778
Runda antrenare  3 scor partial predictie:  0.7345679012345679
Runda antrenare  4 scor partial predictie:  0.7592592592592593
200 auto 0.1 entropy
Runda antrenare  0 scor partial predictie:  0.7962962962962963
Runda antrenare  1 scor partial predictie:  0.790123456790

Runda antrenare  0 scor partial predictie:  0.7592592592592593
Runda antrenare  1 scor partial predictie:  0.7962962962962963
Runda antrenare  2 scor partial predictie:  0.7530864197530864
Runda antrenare  3 scor partial predictie:  0.7716049382716049
Runda antrenare  4 scor partial predictie:  0.7839506172839507
100 auto 0.3 gini
Runda antrenare  0 scor partial predictie:  0.8024691358024691
Runda antrenare  1 scor partial predictie:  0.7654320987654321
Runda antrenare  2 scor partial predictie:  0.8024691358024691
Runda antrenare  3 scor partial predictie:  0.7530864197530864
Runda antrenare  4 scor partial predictie:  0.7530864197530864
100 auto 0.3 entropy
Runda antrenare  0 scor partial predictie:  0.7098765432098766
Runda antrenare  1 scor partial predictie:  0.7592592592592593
Runda antrenare  2 scor partial predictie:  0.7407407407407407
Runda antrenare  3 scor partial predictie:  0.7901234567901234
Runda antrenare  4 scor partial predictie:  0.8271604938271605
100 auto 0.4 gin

Runda antrenare  3 scor partial predictie:  0.7901234567901234
Runda antrenare  4 scor partial predictie:  0.7654320987654321
500 auto 0.5 gini
Runda antrenare  0 scor partial predictie:  0.7592592592592593
Runda antrenare  1 scor partial predictie:  0.8333333333333334
Runda antrenare  2 scor partial predictie:  0.8024691358024691
Runda antrenare  3 scor partial predictie:  0.7469135802469136
Runda antrenare  4 scor partial predictie:  0.7407407407407407
500 auto 0.5 entropy
Runda antrenare  0 scor partial predictie:  0.7839506172839507
Runda antrenare  1 scor partial predictie:  0.7839506172839507
Runda antrenare  2 scor partial predictie:  0.7530864197530864
Runda antrenare  3 scor partial predictie:  0.7962962962962963
Runda antrenare  4 scor partial predictie:  0.7654320987654321


KeyError: "['final_test_score'] not in index"

In [57]:
display(df_final[['final_test_fitting_score','final_test_prediction_score','n_estimators','max_features','max_samples','criterion']])

,final_test_fitting_score,final_test__prediction_score,n_estimators,max_features,max_samples,criterion
0,0.815642,0.619048,100,auto,0.1,gini
1,0.837989,0.601732,100,auto,0.1,gini
2,0.817505,0.588745,100,auto,0.1,gini
3,0.810056,0.601732,100,auto,0.1,gini
4,0.821229,0.523810,100,auto,0.1,gini


## Exemplu folosind GridSearchCV


Metodele de validare incrucisata oferite de sklearn pentru estimatori au multe avantaje:
- cod compact
- paralelizare by default
- o larga gama de modele pot fi antrenate astfel

De retinut: pe cat posibil evitați pentru moment atribute derivate la nivel de eșantion pentru că aceste atribute nu pot fi construite decăt prin crearea unei linii de antrenament (pipeline) care să conțină toți pașii de transformare a datelor și care poate fi folosit ca atare de GridSearchCV.

NOTA BENE: Pentru unele modele există implementări dedicate de CV, așa cum am văzut la LassoCV sau RidgeCV.

Vom vorbi semestrul următor de linii de asamblare, în contextul construirii unei linii de antrenare/re-antrenare complete, precum și a unui mod de utilizare a modelului salvat. Cu acest prilej vom vedea cum putem folosi GridSearchCV

In [ ]:
def antrenament_scikitlearn(X,y):
    
    rfc=RandomForestClassifier(random_state=42)
    param_grid = {
                    'n_estimators': [100, 200, 500, 1000],
                    'max_features': ['auto'],
                    'max_samples' : [0.1, 0.2, 0.3, 0.4, 0.5 ]#,0.6 ,0.7, 0.8, 0.9] ,
                    'criterion' :['gini', 'entropy']
                 }
    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, n_jobs = 4)
    CV_rfc.fit(X, y)
    
    return CV_rfc.best_params_ , CV_rfc.best_estimator_

results_CV_paralel = []

target_col = 'label'
X = pima[[c for c in pima.columns if c != target_col]]
y= pima[target_col]
# cateva iteratii 
for i in range(5):
    
    print("Iteration {0} of 5".format(i))
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.3, stratify = y) # 70% training and 30% test
    
    params,final_model = antrenament_scikitlearn(X_train.to_numpy(), y_train.to_numpy())
    
    # nu este necesar sa mai instantiem un model folosing cei mai buni parametrii - exact acest obiect este rezultatul
    # sesiunii de antrenament
    final_model.fit(X_train, y_train)
    y_test_pred = final_model.predict(X_test)
    
    final_model.fit(X_train[[c for c in pima.columns if c!= target_col]], y_train)
    y_train_pred = final_model.predict(X_train[[c for c in pima.columns if c!= target_col]])
    y_test_pred = final_model.predict(X_test[[c for c in pima.columns if c!= target_col]])
    params['final_test_fitting_score']= accuracy(y_true = y_train, y_pred=y_train_pred)
    params['final_test__prediction_score']= accuracy(y_true = y_test, y_pred=y_test_pred)
    results_CV_paralel.append(params)
    

df_final = pd.DataFrame(results_CV_paralel)

In [54]:
display(df_final[['final_test_fitting_score','final_test__prediction_score','n_estimators','max_features','max_samples','criterion']])

,final_test_fitting_score,final_test__prediction_score,n_estimators,max_features,max_samples,criterion
0,0.832402,0.731602,100,auto,0.1,entropy
1,0.826816,0.748918,100,auto,0.1,gini
2,0.810056,0.761905,100,auto,0.1,gini
3,0.811918,0.757576,100,auto,0.1,entropy
4,0.821229,0.774892,100,auto,0.1,entropy
